In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary

from scipy import ndimage
import numpy as np
import PIL
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

from utilities.common_utils import *
from utilities.landmark_utils import *
from utilities.plotting import *
from models import ResUNet, UNet


## Get dataset
We will use the 'A Grand Challenge for Automated Detection of Anatomical Landmarks and Analysis for Diagnosis in Cephalometric X-ray Image' dataset, see  http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html

Download 'A Grand Challenge for Automated Detection of Anatomical Landmarks and Analysis for Diagnosis in Cephalometric X-ray Image
data, see  http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html

In [3]:
path = Path('./data'); path.mkdir(exist_ok=True)

### Download

In [4]:
URL = 'https://ndownloader.figshare.com/articles/3471833?private_link=37ec464af8e81ae6ebbf'
!cd {path}
!curl -o {path}/dataset.zip {URL}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1042M  100 1042M    0     0  8556k      0  0:02:04  0:02:04 --:--:-- 8524k     0  7679k      0  0:02:19  0:01:01  0:01:18 9351k


### Unpack

In [14]:
!unzip -d {path} {path}/dataset.zip

In [15]:
!unar -q -f -o {path} {path}/AnnotationsByMD.rar
!unar -q -f -o {path} {path}/EvaluationCode.rar
!unar -q -f -o {path} {path}/RawImage.rar

### Create subsampled training and test datasets

In [20]:
test_orig_path = path/'RawImage/Test1Data'
train_orig_path = path/'RawImage/TrainingData'

IMAGE_SIZE = 128

In [21]:
def resize(fpath, img_size, resized_dir):
    PIL.Image.open(fpath).resize((img_size, img_size)).save(resized_dir/fpath.name)

Resize test images

In [22]:
test_path = path/f'images/{IMAGE_SIZE}/test1'; test_path.mkdir(parents=True, exist_ok=True)
l_resize = lambda fpath : resize(fpath, IMAGE_SIZE, test_path)
with ThreadPoolExecutor(16) as e: e.map(l_resize, list_files(test_orig_path))

Resize train images

In [23]:
train_path = path/f'images/{IMAGE_SIZE}/train'; train_path.mkdir(exist_ok=True) 
l_resize = lambda fpath : resize(fpath, IMAGE_SIZE, train_path)
with ThreadPoolExecutor(16) as e: e.map(l_resize, list_files(train_orig_path))